# load model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from toolkit.nlp import NLPTrainingConfig
path = './outputs/optimal_checkpoint_epoch2/'
config = NLPTrainingConfig.load(path)

model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, torch_dtype='auto')
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

/usr/local/python3.11.2/lib/python3.11/site-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")


2023-10-18 15:08:23,253 <INFO> Configuration: 👻 Custom attributes:
2023-10-18 15:08:23,254 <INFO> Configuration:    cache_dir=None
2023-10-18 15:08:23,254 <INFO> Configuration:    model_revision=main
2023-10-18 15:08:23,255 <INFO> Configuration:    use_fast_tokenizer=True
2023-10-18 15:08:23,256 <INFO> Configuration:    use_auth_token=False
2023-10-18 15:08:23,256 <INFO> Configuration:    preprocessing_num_workers=8
2023-10-18 15:08:23,257 <INFO> Configuration:    max_seq_length=2048
2023-10-18 15:08:23,257 <INFO> Configuration:    deepspeed_config=./ds_zero3_offload.hjson
2023-10-18 15:08:23,258 <INFO> Configuration:    generate_config_file=generate_config.json
2023-10-18 15:08:23,258 <INFO> Configuration:    re_gen_num=2
2023-10-18 15:08:23,259 <INFO> Configuration:    total_steps_num=470
2023-10-18 15:08:23,260 <INFO> Configuration:    steps_per_epoch=94


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import pandas as pd
df = pd.read_json(config.val_file_path, lines=True)
df.head(2)

,instruction,input,output,query
0,,徐怀钰参加综艺节目乘风后，引起了广泛关注。她曾经经历过家庭贫困和爷爷的心理问题，复出时效果并...,\n徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n徐怀钰的经历充满波折，家庭贫困...,None
1,,南京同曦官宣引进法尔和布莱克尼 法尔和布莱克尼加入球队。随着南京同曦更换赞助商之后，球队在引...,同曦男篮宣布引进法尔和布莱克尼，给球队的阵容注入了新的活力。他们的加入有望帮助球队突破历史，...,媒体人谈同曦新双外：针对性很强


In [4]:
# import pandas as pd
# df = pd.read_json("data/hot_finetune_data/train/all.json", lines=True)
# df.head(2)

In [19]:
from build_dataset import prompt_transfer
import torch
torch.cuda.set_device(0)

model.cuda()
for idx, row in df.iterrows():
    inputs = prompt_transfer(row['input'], row['output'])
    print(inputs)
    batch = tokenizer(inputs, return_tensors='pt')
    batch = {key: value.cuda() for key, value in batch.items()}
    outputs = model.generate(**batch, **config.generate_kwargs)
    texts = []
    for idx, output in enumerate(outputs):
        texts.append(tokenizer.decode(output[batch['input_ids'][idx].size(0):], skip_special_tokens=True))
    # texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    break

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
给定文本：
徐怀钰参加综艺节目乘风后，引起了广泛关注。她曾经经历过家庭贫困和爷爷的心理问题，复出时效果并不理想。然而，她在乘风中逐渐找回了自我，与秋瓷炫等人成为了好友，并和黄丽玲合作。虽然她不是王心凌，但乘风提供了一个机会，让她重新找回自我，比输赢更重要。徐怀钰的经历令人心疼。她从小家庭贫困，爷爷对她不好，留下了永远的心理创伤。这些年，她虽然一直在努力复出，但效果都不好。在参加乘风时，一公因路灯下的小姑娘被抨击而难过。好在Ella等人一起安慰她，她才挺过来。五公时，慢热的徐怀钰终于找到了感觉。秋瓷炫都为徐怀钰的变化感觉高兴，看得出她开始享受舞台了。这次居然主动和黄丽玲合作，要知道，这两位实力唱将合作，应该会很有很多火花。黄丽玲也很期待。徐怀钰不是王心凌，但她的复出并不仅仅是为了赢。对于粉丝来说，乘风只是一个综艺，能让徐怀钰找回自我，比什么输赢都重要。乘风让徐怀钰逐渐敞开心扉，与秋瓷炫等人成为了好友，也让她重新找到了表演的乐趣。她还和黄丽玲合作过，黄丽玲也很期待。有些人可能对徐怀钰的表现有所期待，毕竟她曾经是一位著名歌手，但输赢对她来说似乎已经不再重要了。
生成300字左右视频文案。

### Response: 


In [22]:
print(texts[0])


徐怀钰参加综艺节目《乘风》引发关注，她曾是家庭贫困和爷爷心理问题的受害者，复出效果不佳。
徐怀钰从小家庭贫困，爷爷对她不好，留下了永远的心理创伤。虽然她一直努力复出，但效果都不好。参加《乘风》时，一公因路灯下的小姑娘被抨击而难过，好在Ella等人安慰她，她才挺过来。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫都为她变化感到高兴，她开始享受舞台了。甚至主动与黄丽玲合作，要知道，这两位实力唱将合作，应该会很有很多火花。黄丽玲也很期待。
徐怀钰虽然不是王心凌，但她的复出并不仅仅是为了赢。对于粉丝来说，《乘风》只是一个综艺，能让徐怀钰找回自我，比什么输赢都重要。


In [21]:
print(row['output'])


徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。
徐怀钰的经历充满波折，家庭贫困、爷爷的心理问题，这些年一直在努力付出。虽然在《乘风》的舞台上复出效果不理想，但我们也看到了徐怀钰逐渐找回自我，这比比赛的输赢更重要。在《乘风》中，她与秋瓷炫、黄丽玲等人成为了好友，并携手合作。虽然她没有王心凌的机遇，但《乘风》也为她提供了一个重要的机会。她因一公被抨击而感到难过，好在有Ella等人的安慰与支持，让她挺过了难关。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫也为她的变化感到高兴，看出她开始享受舞台了。这次，徐怀钰主动和黄丽玲合作，无疑将带来许多火花。
作为徐怀钰复出的舞台，《乘风》让徐怀钰找回终于找回了自我。让我们一起期待徐怀钰更多精彩表现！


# test evaluator

In [ ]:
from toolkit.training import Evaluator
from load_data_fn import load_data_fn
from toolkit.enums import Split
from toolkit.nlp import TextDataset
from toolkit.metric import rouge, bleu
def eval_callback(all_labels, all_logits, mean_loss):
    metric = rouge(all_logits, all_labels, "zh", ("rouge1", "rouge2", "rougeL"))
    metric.update(bleu(all_logits, all_label, 'zh', ("bleu1", "bleu2", "bleu3", "bleu4")))
    return metric, all_labels, all_logits

config.infer_batch_size=8
dataset = TextDataset.from_file(config.val_file_path, tokenizer, Split.VALIDATION, config, load_data_fn, use_cache=False, debug=True)
evaluator = Evaluator('generate', Split.VALIDATION, config, model, dataset, eval_callback, tokenizer=tokenizer, extral_args_evaluation=None)
metric, all_label, all_logits = evaluator.eval(1)

In [ ]:
print(metric)
for i in range(8):
    print("-----label------")
    print(all_label[i])
    print("-----pred------")
    print(all_logits[i])
    print('########################################')

# test extral evaluator

In [ ]:
# import json

# with open("generate_config.json", "r", encoding='utf8') as f:
#     generate_config = json.load(f)


# with open("generate_config.json", "w", encoding='utf8') as f:
#     json.dump(generate_config, f, indent=2)

# generate_config


In [4]:
from extral_evaluator import Extral_Evaluator, extra_calculate_metric_callback
from load_data_fn import load_data_fn
from toolkit.enums import Split
from toolkit.nlp import TextDataset
from toolkit.metric import self_bleu
def extra_calculate_metric_callback(all_labels, all_logits):
    return self_bleu(all_logits, 'zh', ("bleu1", "bleu2", "bleu3", "bleu4"), smoothing_level=1), all_labels, all_logits

# config.generate_config_file="generate_config.json"
# config.re_gen_num=2
config.infer_batch_size=8


dataset = TextDataset.from_file(config.val_file_path, tokenizer, Split.VALIDATION, config, load_data_fn, use_cache=False, debug=True)
evaluator = Extral_Evaluator('generate', Split.VALIDATION, config, model, dataset, extra_calculate_metric_callback, tokenizer=tokenizer)
metric, all_label, all_logits  = evaluator.eval(0)

2023-10-18 15:17:41,592 <DEBUG> TextDataset: ⏳ Loading VALIDATION dataset ...
2023-10-18 15:17:41,593 <INFO> TextDataset: Model max length: 4096


Tokenize input texts:   0%|          | 0/4 [00:00<?, ?it/s]

2023-10-18 15:17:41,640 <DEBUG> TextDataset: ⌛ Loading VALIDATION data takes 0.05 sec.
2023-10-18 15:17:41,640 <INFO> TextDataset: Total data: 4
2023-10-18 15:17:41,641 <INFO> TextDataset: Max length of input: 795
2023-10-18 15:17:41,641 <INFO> TextDataset: Max length of label: -1


VALIDATION:   0%|          | 0/2 [00:00<?, ?batch/s]

VALIDATION:   0%|          | 0/2 [00:00<?, ?batch/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
from pprint import pp
print(metric)
for i in range(4):
    print("-----label------")
    pp(all_label[i])
    print("-----pred------")
    for text in all_logits[i]:
        print(text)
        print('--------------------------')
    print('########################################')

{'self-bleu1': 0.40526503186515067, 'self-bleu2': 0.5724364766829015, 'self-bleu3': 0.6763363723189562, 'self-bleu4': 0.7455375995634501}
-----label------
('\n'
 '徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n'
 '徐怀钰的经历充满波折，家庭贫困、爷爷的心理问题，这些年一直在努力付出。虽然在《乘风》的舞台上复出效果不理想，但我们也看到了徐怀钰逐渐找回自我，这比比赛的输赢更重要。在《乘风》中，她与秋瓷炫、黄丽玲等人成为了好友，并携手合作。虽然她没有王心凌的机遇，但《乘风》也为她提供了一个重要的机会。她因一公被抨击而感到难过，好在有Ella等人的安慰与支持，让她挺过了难关。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫也为她的变化感到高兴，看出她开始享受舞台了。这次，徐怀钰主动和黄丽玲合作，无疑将带来许多火花。\n'
 '作为徐怀钰复出的舞台，《乘风》让徐怀钰找回终于找回了自我。让我们一起期待徐怀钰更多精彩表现！')
-----pred------
在《乘风》后，徐怀钰的家庭贫困和爷爷的心理问题引发广泛关注。但是，她在节目中却逐渐找回了自我，与秋瓷炫等人成为了好友，并和黄丽玲合作了一首歌曲。
尽管徐怀钰并非王心凌，但在乘风中她展现出了自我找回的意义，获得了重拾表演的乐趣。她的经历让人心疼，从小家庭贫困、爷爷对她不好，导致她一直有着心理创伤。尽管努力复出，但效果并不理想。直到参加乘风时，因为路灯下的小姑娘被抨击而难过，幸亏Ella等人的支持才让她挺过去。第五届公演中，徐怀钰终于找到了感觉，展现出了自己的特色。秋瓷炫对徐怀钰的变化感到欣喜，认为她开始享受舞台了。此次与黄丽玲合作更是让人期待，两位实力唱将的合作必将产生更多火花。
徐怀钰的复出也许对某些人来说是意料之中，但她的成功却超乎所有人的预期。无论如何，对于粉丝来说，乘风只是一个综艺，能帮徐怀钰找回自我，比什么输赢都重要。让我们一起期待徐怀钰在未来更多的表现吧！你是否赞同徐怀钰的观点？欢迎留言评论分享你的想法！
--------------------------
徐怀钰参加综艺节目《乘风》后，引起广泛关注。她曾经历了家庭贫困和爷爷的心理问题，复出时效

In [8]:
from extral_evaluator import Extral_Evaluator, extra_calculate_metric_callback
from load_data_fn import load_data_fn
from toolkit.enums import Split
from toolkit.nlp import TextDataset
from toolkit.metric import self_bleu
def extra_calculate_metric_callback(all_labels, all_logits):
    return self_bleu(all_logits, 'zh', ("bleu1", "bleu2", "bleu3", "bleu4"), smoothing_level=1), all_labels, all_logits

# config.generate_config_file="generate_config.json"
# config.re_gen_num=2
config.infer_batch_size=8
config.re_gen_num=3

dataset = TextDataset.from_file(config.val_file_path, tokenizer, Split.VALIDATION, config, load_data_fn, use_cache=False, debug=True, part=8)
evaluator = Extral_Evaluator('generate', Split.VALIDATION, config, model, dataset, extra_calculate_metric_callback, tokenizer=tokenizer)
metric, all_label, all_logits  = evaluator.eval(0)

2023-10-18 15:35:08,615 <DEBUG> TextDataset: ⏳ Loading VALIDATION dataset ...
2023-10-18 15:35:08,616 <INFO> TextDataset: Model max length: 4096


Tokenize input texts:   0%|          | 0/8 [00:00<?, ?it/s]

2023-10-18 15:35:08,696 <DEBUG> TextDataset: ⌛ Loading VALIDATION data takes 0.08 sec.
2023-10-18 15:35:08,697 <INFO> TextDataset: Total data: 8
2023-10-18 15:35:08,697 <INFO> TextDataset: Max length of input: 974
2023-10-18 15:35:08,698 <INFO> TextDataset: Max length of label: -1


VALIDATION:   0%|          | 0/4 [00:00<?, ?batch/s]

VALIDATION:   0%|          | 0/4 [00:00<?, ?batch/s]

VALIDATION:   0%|          | 0/4 [00:00<?, ?batch/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
from pprint import pp
print(metric)
for i in range(len(all_label)):
    print("-----label------")
    pp(all_label[i])
    print("-----pred------")
    for text in all_logits[i]:
        print(text)
        print('--------------------------')
    print('########################################')

{'self-bleu1': 0.40122327261079327, 'self-bleu2': 0.5291207586789934, 'self-bleu3': 0.6129560751164513, 'self-bleu4': 0.6714703962216153}
-----label------
('\n'
 '徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n'
 '徐怀钰的经历充满波折，家庭贫困、爷爷的心理问题，这些年一直在努力付出。虽然在《乘风》的舞台上复出效果不理想，但我们也看到了徐怀钰逐渐找回自我，这比比赛的输赢更重要。在《乘风》中，她与秋瓷炫、黄丽玲等人成为了好友，并携手合作。虽然她没有王心凌的机遇，但《乘风》也为她提供了一个重要的机会。她因一公被抨击而感到难过，好在有Ella等人的安慰与支持，让她挺过了难关。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫也为她的变化感到高兴，看出她开始享受舞台了。这次，徐怀钰主动和黄丽玲合作，无疑将带来许多火花。\n'
 '作为徐怀钰复出的舞台，《乘风》让徐怀钰找回终于找回了自我。让我们一起期待徐怀钰更多精彩表现！')
-----pred------
徐怀钰参加综艺节目《乘风》后，引起广泛关注。她曾经历过家庭贫困和爷爷的心理问题，复出时效果不尽如人意。但在《乘风》中，她逐渐找回了自我，与秋瓷炫等人成为好友，并与黄丽玲合作。
她的经历令人心疼。从小家境贫寒，家庭教育不足，导致她无法自信地面对自己的人生。尽管一直努力复出，但结果并不如人意。在参加《乘风》时，第一次公演因为路灯下的小女孩被抨击而难过。幸亏Ella等人的陪伴和支持，她才能坚持下来。到五公时，徐怀钰终于找到舞台的感觉，秋瓷炫也为她的变化感到高兴。她开始享受舞台，展现自己的表演技巧和热情。此次与黄丽玲的合作更是备受期待，相信他们会有很多火花。黄丽玲表示非常期待与徐怀钰的合作，因为他曾经是她的偶像。
徐怀钰的复出之路不易，但她已找回自我，获得了重生的机会。对于粉丝而言，《乘风》只是一档综艺节目，它能给予徐怀钰重拾自我的机会，胜过一切输赢。让我们一起关注她的成长，看她如何在音乐的道路上再次闪耀！你认为徐怀钰能否通过这次复出，重新获得成功？请留言分享你的看法！
--------------------------
徐怀钰参加综艺节目《乘风》后，引起广泛关

In [4]:
import pandas as pd

preds = ['哦', '饿', '啊']
labels = [['求'], ['我'], ['用']]

d = dict(preds=preds, labels=labels )

df = pd.DataFrame.from_dict(d)

In [5]:
df

,preds,labels
0,哦,[求]
1,饿,[我]
2,啊,[用]


In [8]:
df.to_json('test.json', force_ascii=False, indent=2, orient='records')

In [9]:
from pathlib import Path

p = Path('a/b/c.json')

In [14]:
p.parent

PosixPath('a/b')

# just metric

## rouge差异 between temperature is 0.01 and 1

In [27]:
import pandas as pd
from pathlib import Path
from toolkit.metric import rouge

extra_ret = dict()
epoch = 0
evaluator_file_dir = Path("outputs/hot_finetune_data/baichuan-13b-chat/baseline/10/64/0.0001/12190/evaluators/extal_evaluator/")
for epoch in range(6):
    file_path = next(evaluator_file_dir.glob(f"epoch={epoch:03d}*"))
    df = pd.read_json(file_path)
    preds = df['preds'].tolist()
    labels = df['labels'].tolist()
    extra_ret[f"epoch={epoch:03d}"]=rouge(labels, preds, "zh", ("rouge1", "rouge2", "rougeL"))


from pprint import pp
pp(extra_ret)


Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

{'epoch=000': {'rouge1': 0.48512448789551854,
               'rouge2': 0.19590583614384136,
               'rougeL': 0.2470928463153541},
 'epoch=001': {'rouge1': 0.5255091662208239,
               'rouge2': 0.2418336006036649,
               'rougeL': 0.28105681024802226},
 'epoch=002': {'rouge1': 0.5854789253013829,
               'rouge2': 0.31586406849479925,
               'rougeL': 0.3344592328649014},
 'epoch=003': {'rouge1': 0.6083813579753041,
               'rouge2': 0.35394720199595514,
               'rougeL': 0.3661546645841251},
 'epoch=004': {'rouge1': 0.6163252286302546,
               'rouge2': 0.37104262310701114,
               'rougeL': 0.3827449827610205},
 'epoch=005': {'rouge1': 0.631963022518903,
               'rouge2': 0.4012871064284506,
               'rougeL': 0.4083203771927704}}


In [33]:
import pandas as pd
from pathlib import Path
from toolkit.metric import rouge

ret = dict()
epoch = 0
evaluator_file_dir = Path("outputs/hot_finetune_data/baichuan-13b-chat/baseline/10/64/0.0001/12190/evaluators/evaluator/")
for epoch in range(6):
    file_path = next(evaluator_file_dir.glob(f"epoch={epoch:03d}*"))
    df = pd.read_json(file_path)
    preds = df['preds'].tolist()
    labels = df['labels'].tolist()
    ret[f"epoch={epoch:03d}"]=rouge(labels, preds, "zh", ("rouge1", "rouge2", "rougeL"))



from pprint import pp
pp(ret)

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

{'epoch=000': {'rouge1': 0.6044565091530482,
               'rouge2': 0.37181247894962627,
               'rougeL': 0.39106747151042026},
 'epoch=001': {'rouge1': 0.5966055582587918,
               'rouge2': 0.3564615677848148,
               'rougeL': 0.3767784895996253},
 'epoch=002': {'rouge1': 0.626522671431303,
               'rouge2': 0.38104504680571455,
               'rougeL': 0.38788659311831},
 'epoch=003': {'rouge1': 0.6363612441346049,
               'rouge2': 0.40268681621334207,
               'rougeL': 0.40422341770802933},
 'epoch=004': {'rouge1': 0.637975925890108,
               'rouge2': 0.4080157083614419,
               'rougeL': 0.4139072469746073},
 'epoch=005': {'rouge1': 0.6330715282432114,
               'rouge2': 0.4122696421885242,
               'rougeL': 0.416988258366473}}


In [34]:
diff_ret = dict()
for key in ret.keys():
    diff_ret[key] = ret[key]-extra_ret[key]
    # print(ret[key])
    # print(extra_ret[key])
pp(diff_ret)

{'epoch=000': {'rouge1': 0.11933202125752962,
               'rouge2': 0.1759066428057849,
               'rougeL': 0.14397462519506615},
 'epoch=001': {'rouge1': 0.07109639203796791,
               'rouge2': 0.1146279671811499,
               'rougeL': 0.09572167935160303},
 'epoch=002': {'rouge1': 0.04104374612992012,
               'rouge2': 0.0651809783109153,
               'rougeL': 0.05342736025340855},
 'epoch=003': {'rouge1': 0.027979886159300804,
               'rouge2': 0.04873961421738693,
               'rougeL': 0.03806875312390423},
 'epoch=004': {'rouge1': 0.021650697259853358,
               'rouge2': 0.03697308525443077,
               'rougeL': 0.03116226421358681},
 'epoch=005': {'rouge1': 0.0011085057243084284,
               'rouge2': 0.010982535760073575,
               'rougeL': 0.008667881173702596}}


## 文心

In [53]:
import pandas as pd

df = pd.read_json("data/eb_val_res.json", lines=True)
labels = pd.read_json("data/hot_finetune_data/val/val.json", lines=True)['output']
df = pd.merge(df, labels, left_index=True, right_index=True)
df.head()

,result1,result2,output
0,\n徐怀钰在乘风中找回自我，和好友合作让观众期待不已！\n\n徐怀钰的经历充满波折，家庭贫困...,\n徐怀钰在乘风中找回自我，与秋瓷炫等好友合作，享受舞台的乐趣。\n\n徐怀钰的经历从小家庭...,\n徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n徐怀钰的经历充满波折，家庭贫困...
1,\n新赛季新气象！南京同曦男篮迎来大动作，官宣两名强援加入球队。\n\n南京同曦男篮在这个赛...,\n新赛季，南京同曦男篮引进了两名强力外援，布莱克尼和法尔。\n\n南京同曦男篮这个赛季终于...,同曦男篮宣布引进法尔和布莱克尼，给球队的阵容注入了新的活力。他们的加入有望帮助球队突破历史，...
2,\n中俄联合空中战略巡航再次震撼来袭！\n\n6月7日，中俄两军在太平洋西部空域，完成第6次...,\n中俄两国空军进行了第6次联合空中战略巡航，这次巡航有什么特别之处？\n\n根据报道，这次...,6月7日，中俄联合空中战略巡航第二阶段任务落幕。在这次空中巡航中，中国空军出动的并非日本和韩...
3,\n电视剧《苍兰诀》迎来开播一周年，这个夏天，一起感受王鹤棣的魅力！\n\n《苍兰诀》成为去...,\n电视剧《苍兰诀》迎来开播一周年，这个夏天，一起感受爱与和平。\n\n《苍兰诀》自去年播出...,王鹤棣在电视剧苍兰诀中的表现备受追捧，他的一句庆祝一周年的微博引发了热议。\n电视剧苍兰诀以...
4,\n苏丹首都喀土穆的武装冲突仍在持续，这场冲突已经造成大量人员伤亡。\n\n苏丹武装部队与快...,\n苏丹首都喀土穆的武装冲突仍在持续，这场冲突已经严重威胁到当地的基础设施和民生。\n\n据...,当地时间8月5日，苏丹首都喀土穆及其周边地区的武装冲突持续，苏丹武装部队进抵喀土穆国际机场附...


In [54]:
df.rename({'output':'labels'}, axis=1, inplace=True)
df['preds'] = [list(x) for x in zip(df['result1'], df['result2'])]
df.drop(['result1', 'result2'], axis=1, inplace=True)
# df.drop('result2', axis=1, inplace=True)
df.head()

,labels,preds
0,\n徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n徐怀钰的经历充满波折，家庭贫困...,[\n徐怀钰在乘风中找回自我，和好友合作让观众期待不已！\n\n徐怀钰的经历充满波折，家庭贫...
1,同曦男篮宣布引进法尔和布莱克尼，给球队的阵容注入了新的活力。他们的加入有望帮助球队突破历史，...,[\n新赛季新气象！南京同曦男篮迎来大动作，官宣两名强援加入球队。\n\n南京同曦男篮在这个...
2,6月7日，中俄联合空中战略巡航第二阶段任务落幕。在这次空中巡航中，中国空军出动的并非日本和韩...,[\n中俄联合空中战略巡航再次震撼来袭！\n\n6月7日，中俄两军在太平洋西部空域，完成第6...
3,王鹤棣在电视剧苍兰诀中的表现备受追捧，他的一句庆祝一周年的微博引发了热议。\n电视剧苍兰诀以...,[\n电视剧《苍兰诀》迎来开播一周年，这个夏天，一起感受王鹤棣的魅力！\n\n《苍兰诀》成为...
4,当地时间8月5日，苏丹首都喀土穆及其周边地区的武装冲突持续，苏丹武装部队进抵喀土穆国际机场附...,[\n苏丹首都喀土穆的武装冲突仍在持续，这场冲突已经造成大量人员伤亡。\n\n苏丹武装部队与...


In [55]:
from toolkit.metric import self_bleu, rouge

labels = df['labels']
preds = df['preds']
rouge_ret = rouge(labels, preds, "zh", ("rouge1", "rouge2", "rougeL"))
self_bleu_ret = self_bleu(preds, "zh", ("bleu1", "bleu2", "bleu3", "bleu4"), smoothing_level=1)

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

In [57]:
from pprint import pp
pp(rouge_ret)
pp(self_bleu_ret)

{'rouge1': 0.5758505158592016,
 'rouge2': 0.3321792743226979,
 'rougeL': 0.3297982896522929}
{'self-bleu1': 0.36605799082564805,
 'self-bleu2': 0.4788424714944893,
 'self-bleu3': 0.5493356164814367,
 'self-bleu4': 0.5986037954323528}


In [58]:
import pandas as pd
from pathlib import Path
from toolkit.metric import rouge

extra_ret_rouge = dict()
extra_ret_self_bleu = dict()
epoch = 0
evaluator_file_dir = Path("outputs/hot_finetune_data/baichuan-13b-chat/baseline/10/64/0.0001/12190/evaluators/extal_evaluator/")
for epoch in range(6):
    file_path = next(evaluator_file_dir.glob(f"epoch={epoch:03d}*"))
    df = pd.read_json(file_path)
    preds = df['preds'].tolist()
    labels = df['labels'].tolist()
    extra_ret_rouge[f"epoch={epoch:03d}"]=rouge(labels, preds, "zh", ("rouge1", "rouge2", "rougeL"))
    extra_ret_self_bleu[f"epoch={epoch:03d}"]=self_bleu(preds, "zh", ("bleu1", "bleu2", "bleu3", "bleu4"), smoothing_level=1)


Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

...

  0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

Calculating rouge:   0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

In [65]:
print("# -------------------rouge-------------------------")
print("文心")
tmp = (rouge_ret*100)
tmp.round(2)
pp(tmp)
print("baseline")
tmp = extra_ret_rouge.copy()
for key, value in tmp.items():
    tmp[key] = value*100
    tmp[key].round(2)
pp(tmp)


# -------------------rouge-------------------------
文心
{'rouge1': 57.59, 'rouge2': 33.22, 'rougeL': 32.98}
baseline
{'epoch=000': {'rouge1': 48.51, 'rouge2': 19.59, 'rougeL': 24.71},
 'epoch=001': {'rouge1': 52.55, 'rouge2': 24.18, 'rougeL': 28.11},
 'epoch=002': {'rouge1': 58.55, 'rouge2': 31.59, 'rougeL': 33.45},
 'epoch=003': {'rouge1': 60.84, 'rouge2': 35.39, 'rougeL': 36.62},
 'epoch=004': {'rouge1': 61.63, 'rouge2': 37.1, 'rougeL': 38.27},
 'epoch=005': {'rouge1': 63.2, 'rouge2': 40.13, 'rougeL': 40.83}}


In [68]:
print("# -------------------self-bleu-------------------------")
print("文心")
tmp = ((1-self_bleu_ret)*100)
tmp.round(2)
pp(tmp)
print("baseline")
tmp = extra_ret_self_bleu.copy()
for key, value in tmp.items():
    tmp[key] = (1-value)*100
    tmp[key].round(2)
pp(tmp)

# -------------------self-bleu-------------------------
文心
{'self-bleu1': 63.39,
 'self-bleu2': 52.12,
 'self-bleu3': 45.07,
 'self-bleu4': 40.14}
baseline
{'epoch=000': {'self-bleu1': 44.72,
               'self-bleu2': 27.89,
               'self-bleu3': 18.01,
               'self-bleu4': 12.24},
 'epoch=001': {'self-bleu1': 50.57,
               'self-bleu2': 33.65,
               'self-bleu3': 23.73,
               'self-bleu4': 17.75},
 'epoch=002': {'self-bleu1': 56.56,
               'self-bleu2': 42.05,
               'self-bleu3': 33.0,
               'self-bleu4': 27.07},
 'epoch=003': {'self-bleu1': 58.31,
               'self-bleu2': 45.67,
               'self-bleu3': 37.74,
               'self-bleu4': 32.38},
 'epoch=004': {'self-bleu1': 59.23,
               'self-bleu2': 48.19,
               'self-bleu3': 41.2,
               'self-bleu4': 36.35},
 'epoch=005': {'self-bleu1': 63.44,
               'self-bleu2': 54.49,
               'self-bleu3': 48.75,
             